In [1]:
import random

class Barco:  # Representa un barco en el juego
    def __init__(self, nombre: str, tamaño: int, posicion: list):
        self.nombre = nombre #nombre de los barcos
        self.tamaño = tamaño
        self.posicion = posicion
        self.golpes = 0  # para contar los golpes

    def recibir_golpe(self, posicion: list):#una lista porque representa las coordenadas
        if posicion in self.posicion:  #si da una coordenada donde esta el barco, suma un golpe, si supera el tamaño del barco, hundir el barco
            self.golpes += 1 #se suma en el contador 
            print(f"Golpe al barco {self.nombre}")
            if self.esta_hundido():
                print(f"Barco {self.nombre} ha sido hundido")
            return True
        return False

    def esta_hundido(self):
        return self.golpes >= self.tamaño  #indica cuando un barco se ha hundido cuando el numero de golpes es >= al barco


class Tablero:
    def __init__(self):
        self.tablero = [[0 for _ in range(10)] for _ in range(10)]  # Crea un tablero 10x10 lleno de ceros
        self.barcos = {}  # Diccionario para almacenar los barcos por nombre

    def colocar_barco(self, nombre_barco, posicion, orientacion, tamaño):
        fila, columna = posicion
        posiciones_barcos = []  # Lista para almacenar la posición de cada barco

       
        if orientacion == "h":
            if columna + tamaño > 10:  # Si el barco se sale del tablero
                return False
            for i in range(tamaño): #Itera tantas veces como celdas ocupa el barco.

                if self.tablero[fila][columna + i] != 0:  #si ya hay un barco en esa posición
                    return False
                posiciones_barcos.append((fila, columna + i))
            for fila, columna in posiciones_barcos:
                self.tablero[fila][columna] = "B"  # Marcar con "B" para representar un barco

       
        elif orientacion == "v":
            if fila + tamaño > 10:
                return False
            for i in range(tamaño):
                if self.tablero[fila + i][columna] != 0:
                    return False
                posiciones_barcos.append((fila + i, columna))
            for fila, columna in posiciones_barcos:
                self.tablero[fila][columna] = "B"

        # Crear el barco y añadirlo al diccionario
        self.barcos[nombre_barco] = Barco(nombre_barco, tamaño, posiciones_barcos)
        return True

    def mostrar_tablero(self):
        for fila in self.tablero:
            print(" ".join(str(celda) for celda in fila))  #recorre el tablero convirtiendolo en una cadena y lo une con un espacio

    def recibir_ataque(self, fila, columna):
        #verifica si da al barco y lo marca como golpeado
        if self.tablero[fila][columna] == "B":
            self.tablero[fila][columna] = "X"  # golpe
            print("¡Impacto!")
            return True
        elif self.tablero[fila][columna] == "X": #comprueba si ya se ha dado a esa coordenada
            print("Ya golpeaste esta posición.")
            return False
        else:
            self.tablero[fila][columna] = "O"  # Agua 
            print("Has fallado.")
            return False

    def todos_barcos_hundidos(self):
        return all(barco.esta_hundido() for barco in self.barcos.values())  # Devuelve True si todos los barcos están hundidos


class Jugador:
    def __init__(self, nombre):
        self.nombre = nombre #nombre del jugador
        print(f"Bienvenido {self.nombre} al juego de hundir la flota!")

    def atacar(self, oponente, fila, columna):
        resultado = oponente.tablero.recibir_ataque(fila, columna) #utiliza el metodo para recibir el ataque del oponente
        print(f"{self.nombre} ha atacado a ({fila}, {columna}) con resultado: {'Impacto' if resultado else 'Fallo'}")
        return resultado


class Juego:
    def __init__(self, jugador1, jugador2):
        self.jugador1 = jugador1
        self.jugador2 = jugador2
        self.tablero1 = Tablero()
        self.tablero2 = Tablero()
        self.turnoactual = jugador1  # Empieza el juego con este turno

    def colocar_barcos(self):
        barcos = {
            "portaaviones": 5,
            "acorazado": 4,
            "crucero": 3,
            "submarino": 3,
            "destructor": 2
        }
        print(f"Colocando barcos para {self.jugador1.nombre}")
        for nombre, tamaño in barcos.items():
            while True:
                fila = random.randint(0, 9)
                columna = random.randint(0, 9)
                orientacion = random.choice(["h", "v"])
                if self.tablero1.colocar_barco(nombre, (fila, columna), orientacion, tamaño):
                    break # valida si los rangos estan bien

        print(f"Colocando barcos para {self.jugador2.nombre}")
        for nombre, tamaño in barcos.items():
            while True:
                fila = random.randint(0, 9)
                columna = random.randint(0, 9)
                orientacion = random.choice(["h", "v"])
                if self.tablero2.colocar_barco(nombre, (fila, columna), orientacion, tamaño):
                    break # valida si los rangos estan bien para jugador 2

    def realizar_ataque(self, tablero):
        while True:
            try:
                fila = int(input("Ingrese una fila a la que atacar (0-9): "))
                columna = int(input("Ingrese una columna a la que atacar (0-9): "))
                if 0 <= fila <= 9 and 0 <= columna <= 9:
                    break
                else:
                    print("Coordenadas fuera del rango.")
            except ValueError:
                print("Debe ingresar coordenadas válidas.")
        
        return tablero.recibir_ataque(fila, columna) #registrar ataque 

    def iniciar_juego(self):
        print("¡Comienza el juego de hundir la flota!")
        while True:
            print(f"Turno de {self.turnoactual.nombre}")
            tablero_oponente = self.tablero2 if self.turnoactual == self.jugador1 else self.tablero1
            
            # Mostrar el tablero del oponente
            print("Tablero del oponente:")
            tablero_oponente.mostrar_tablero()
            
            # Realizar el ataque
            self.realizar_ataque(tablero_oponente)
            
            # Verificar si todos los barcos del oponente han sido hundidos
            if self.finalizar_juego():
                break
            
            # Cambiar de turno
            self.cambiar_turno()

    def finalizar_juego(self):
        if self.tablero1.todos_barcos_hundidos():
            print(f"{self.jugador2.nombre} ha ganado!")
            return True
        elif self.tablero2.todos_barcos_hundidos():
            print(f"{self.jugador1.nombre} ha ganado!")
            return True
        return False

    def cambiar_turno(self):
        self.turnoactual = self.jugador1 if self.turnoactual == self.jugador2 else self.jugador2


# Inicialización de los jugadores y del juego
jugador1 = Jugador("Isabel")
jugador2 = Jugador("Jugador 2")
juego = Juego(jugador1, jugador2)
juego.colocar_barcos()
juego.iniciar_juego()


Bienvenido Isabel al juego de hundir la flota!
Bienvenido Jugador 2 al juego de hundir la flota!
Colocando barcos para Isabel
Colocando barcos para Jugador 2
¡Comienza el juego de hundir la flota!
Turno de Isabel
Tablero del oponente:
0 0 0 0 0 0 0 0 0 0
0 B B B B B B 0 0 0
0 B B B 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 B B B B B 0 0 0 0
0 0 0 0 B 0 0 0 0 0
0 0 0 0 B 0 0 0 0 0
0 0 0 0 B 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
Has fallado.
Turno de Jugador 2
Tablero del oponente:
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 B 0 0 0 0 0 0 0 B
0 B 0 0 0 0 0 B 0 B
0 0 0 B B B 0 B 0 B
0 0 0 0 0 0 0 B 0 B
0 0 0 0 0 0 0 B 0 0
0 0 0 B 0 0 0 B 0 0
0 0 0 B 0 0 0 0 0 0
0 0 0 B 0 0 0 0 0 0
Has fallado.
Turno de Isabel
Tablero del oponente:
0 0 0 0 0 0 0 0 0 0
0 B B B B B B 0 0 0
0 B B B 0 0 0 0 0 0
0 0 0 0 O 0 0 0 0 0
0 B B B B B 0 0 0 0
0 0 0 0 B 0 0 0 0 0
0 0 0 0 B 0 0 0 0 0
0 0 0 0 B 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0
¡Impacto!
Turno de Jugador 2
Tablero del oponente:
0 0 0 0 0